# Creating a conda package from a Modelica Model
This example illustrates how to develop, run, test(, repeat) a Modelica model in a (local) pandas/pyspark environment that can be deployed in a cluster as a conda package for large scale analytics.
- Build and compile the model once
- Make it a conda package
- Call the compiled model with different input data
- Use it on scale. Change parameters only during runtime.

See https://openmodelica.org/doc/OpenModelicaUsersGuide/v1.11.0/ompython.html

In [ ]:
import pandas as  pd
import os
import sys
import json
import tempfile
import importlib

### temporarly adding for dev

In [ ]:
# Finds the package in the repo instead of the installed one
sys.path.insert(0, '../../src')
from OMPython import ModelicaSystemRunner

def instantiatemodel(modelName, use_local=True, force_executable_path=None):
    mod = ModelicaSystemRunner(
            # fileName=fn,
            modelname=modelName,
            runpath='dev_interactive_omc',
        )
    return mod
# mod = BouncingBall.instantiatemodel('./BouncingBall')
mod = instantiatemodel(modelName='EDrives.Examples.DCDC.DC_Drive_Continuous')

## Parametric simulation without OMC
https://openmodelica.org/doc/OpenModelicaUsersGuide/latest/scripting_api.html#simulation-parameter-sweep

In [ ]:
from OMPython import ModelicaSystemRunner

In [ ]:
from MM import EDrives

In [ ]:
mod = EDrives.instantiatemodel("EDrives.Examples.DCDC.DC_Drive_Continuous")  # EDrives.Examples.DCDC.DC_Drive_Switching"
mod.getParameters(['dMax', 'dMin', 'tauNominal', 'dcpmData'])

In [ ]:
p = pd.to_numeric(pd.Series(mod.getParameters()), errors='coerce')

In [ ]:
p['dcpmData.VaNominal'] 

In [ ]:
dMin = float(mod.getParameters(["dMin"])[0])
dMax = 1.0 - dMin
tauNominal=p['dcpmData.VaNominal'] * p['dcpmData.IaNominal'] / p['dcpmData.wNominal']  # The mode uses ViNominal

In [ ]:
ts_df1 = pd.DataFrame(columns=[
    'time', 'value'], data=[
    [0.0, 0.5],
    [3.0, 0.5],
    [4.0, dMax],
    [12.0, dMax],
    [13.0, dMin],
    [21.0, dMin],
    [22.0, 0.5],
    [24.0, 0.5],    
])
ts_df1['input'] = 'dutyCycle_series'
ts_df2 = pd.DataFrame(columns=[
    'time', 'value'], data=[
    [0.0, 0.0],
    [6.0, 0.0],
    [7.0, -tauNominal],
    [9.0, -tauNominal],
    [10.0, tauNominal],
    [15.0, tauNominal],
    [16.0, -tauNominal],
    [18.0, -tauNominal],
    [19.0, 0.0],
    [24.0, 0.0],
])
ts_df2['input'] = 'loadTorque_Series'
ts_df = pd.concat([ts_df1, ts_df2], ignore_index=True)
# ts_df['run_key'] = 'r1'
ts_df = ts_df.pivot(index='time', columns='input', values='value').ffill().reset_index()#.drop('input', axis=1)
ts_df['run_key'] = 'r1'
#ts_df.to_csv('test.csv', sep=',', line_terminator=',\n')

In [ ]:
ts_df['modifiers'] = json.dumps({'sim_options_override': {'stopTime': 3.0,}})

### Sequential execution

In [ ]:
#mod.getContinuous()  # list of states

In [ ]:
from ModelicaRuntimeTools import run_sim_parametric, dymat2pandas, get_sim_dist_func

In [ ]:
res_pars = ['dcpm.wMechanical', 'constantVoltage.i', 'dcpm.brush.i', 'torque.tau', 'hBridge.ref']

In [ ]:
ts_df.drop(columns=['modifiers'], inplace=True)

In [ ]:
ts_df

In [ ]:
#%%time
ts_all_df = ts_df.groupby(['run_key']).apply(
        get_sim_dist_func(EDrives, 'EDrives.Examples.DCDC.DC_Drive_Continuous', res_vars=res_pars, step_divider=5,
        modelicaSystemRunner=mod,
        ), include_groups=True,
    )


In [ ]:
ts_sim_df = ts_all_df

In [ ]:
ts_sim_df

In [ ]:
mod.getOptimizationOptions()

In [ ]:
mod.getSimulationOptions()

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for n, run_key in enumerate(ts_df['run_key'].unique()):
    #ts_run_sim_pdf = ts_sim_sdf.where(F.col('run_key') == run_key).toPandas()
    ts_run_sim_pdf = ts_sim_df[ts_sim_df['run_key'] == run_key]
    plt.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['dcpm.wMechanical'])
plt.show()

In [ ]:
for n, run_key in enumerate(ts_df['run_key'].unique()):
    #ts_run_sim_pdf = ts_sim_sdf.where(F.col('run_key') == run_key).toPandas()
    ts_run_sim_pdf = ts_sim_df[ts_sim_df['run_key'] == run_key]
    plt.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['dcpm.brush.i'])
    #plt.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['constantVoltage.i'])
plt.show()

### Parallel execution with Spark

In [ ]:
ts_sdf = spark.createDataFrame(ts_df)
ts_sdf.show(5)

### Parallel execution

In [ ]:
from pyspark.sql import functions as F, types as T

In [ ]:
from ModelicaRuntimeTools import addpymodules

In [ ]:
sc = spark.sparkContext

In [ ]:
addpymodules(['../../src/DyMat'], 'mdymat.zip', sc=sc, dironly=True)
addpymodules(['../../src/OMPython'], 'mOMPython.zip', sc=sc, dironly=True)

In [ ]:
# Run the model wrapper module in the current namespace
modelwrapper_pyfile = EDrives.__file__

In [ ]:
%run -i $modelwrapper_pyfile

In [ ]:
[T.StructField(p, T.DoubleType(), True) for p in res_pars]

In [ ]:
from pyspark.sql import types as T, functions as F
res_schema = T.StructType([
    T.StructField("time", T.DoubleType(), True)] + [
        T.StructField(p, T.DoubleType(), True) for p in res_pars
    ] + [
    T.StructField("run_key", T.StringType(), True),
])
# Running the parametric simulation

ts_sim_d = {}
ts_sim_d['Switching'] = ts_sdf.groupby(['run_key']).applyInPandas(
        get_sim_dist_func(EDrives, 'EDrives.Examples.DCDC.DC_Drive_Switching', res_vars=res_pars, use_local=False), schema=res_schema,
    ).cache()
ts_sim_d['Continuous'] = ts_sdf.groupby(['run_key']).applyInPandas(
        get_sim_dist_func(EDrives, 'EDrives.Examples.DCDC.DC_Drive_Continuous', res_vars=res_pars, use_local=False), schema=res_schema,
    ).cache()

In [ ]:
ts_sim_d['Continuous'].show()

In [ ]:
runs_list = [r[0] for r in ts_sdf.select('run_key').distinct().toLocalIterator()]

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%%time
plt.figure().clear()
for n, run_key in enumerate(runs_list):
    fig = plt.figure(n)
    ax1 = fig.add_subplot(411)
    ax2 = fig.add_subplot(412, sharex=ax1)
    ax3 = fig.add_subplot(413, sharex=ax1)
    ax4 = fig.add_subplot(414, sharex=ax1)
    fig.tight_layout()
    ax1.set_title('hBridge.ref')
    wMechanical_h_list = []
    i_h_list = []
    for variant in ts_sim_d.keys():
        ts_run_sim_pdf = ts_sim_d[variant].where(F.col('run_key') == run_key).toPandas()
        wMechanical_h, = ax3.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['dcpm.wMechanical'], label=variant)
        i_h, = ax4.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['dcpm.brush.i'], label=variant)
        wMechanical_h_list.append(wMechanical_h)
        i_h_list.append(i_h)
    hBridge_ref, = ax1.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['hBridge.ref'], label='hBridge.ref')
    tau_ref, = ax2.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['torque.tau'], label='tau')
    ax3.legend(handles=wMechanical_h_list)
    ax4.legend(handles=i_h_list)
    fig.show()